## santander

In [24]:
# initialization

%reset -f

import sys

import numpy as np, pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from scipy.stats import kurtosis

# ignore warnings (only if you are the kind that would code when the world is burning)
import warnings
warnings.filterwarnings('ignore')

# some options
MAX_EVALS=5
randomseed = 1 # the value for the random state used at various points in the pipeline
pd.options.display.max_rows = 10 # specify if you want the full output in cells rather the truncated list
pd.options.display.max_columns = 200

# to display multiple outputs in a cell without usin print/display
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# display wd files
import os as os
print('folder files: ', os.listdir(), '\n')
print('envir variables: ')
%who

from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from sklearn.ensemble import GradientBoostingClassifier

from hyperopt import hp, tpe, STATUS_OK, fmin, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample

from sklearn.tree import DecisionTreeClassifier, export_graphviz

folder files:  ['.ipynb_checkpoints', 'dt.dot', 'sample_submission.csv', 'san.ipynb', 'san.pkl', 'sub.csv', 'test.csv', 'train.csv'] 

envir variables: 
MAX_EVALS	 kurtosis	 np	 os	 pd	 randomseed	 sklearn	 sys	 train_test_split	 
warnings	 


In [25]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [26]:
# ## input

# train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test.csv')
# ss = pd.read_csv('./sample_submission.csv')

# train_res = train.target.values
# train.drop(columns=['target', 'ID_code'], inplace=True)
# test.drop(columns=['ID_code'], inplace=True)

In [27]:
# san = open('./san.pkl','wb')
# pickle.dump(train, san)
# pickle.dump(test, san)
# pickle.dump(train_res, san)
# pickle.dump(ss, san)
# san.close()

# load backup
san = open('./san.pkl', 'rb')
train = pickle.load(san)
test = pickle.load(san)
train_res = pickle.load(san)
ss = pickle.load(san)
san.close()

In [28]:
def mmm_feats(df):
    df['max'] = df.max(axis=1)
    df['min'] = df.min(axis=1)
    df['mean'] = df.mean(axis=1)
    df['median'] = df.median(axis=1)
    df['std'] = df.std(axis=1)
    df['kurt'] = df.kurtosis(axis=1)
    df['sum'] = df.sum(axis=1)
    df['skew'] = df.skew(axis=1)
    return df

train = mmm_feats(train)
test = mmm_feats(test)

In [29]:
# from sklearn.tree import _tree

# def tree_to_code(tree, feature_names):
#     tree_ = tree.tree_
#     feature_name = [
#         feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
#         for i in tree_.feature
#     ]
#     print("def tree({}):".format(", ".join(feature_names)))

#     def recurse(node, depth):
#         indent = "  " * depth
#         if tree_.feature[node] != _tree.TREE_UNDEFINED:
#             name = feature_name[node]
#             threshold = tree_.threshold[node]
#             print("{}if {} <= {}:".format(indent, name, threshold))
#             recurse(tree_.children_left[node], depth + 1)
#             print("{}else:  # if {} > {}".format(indent, name, threshold))
#             recurse(tree_.children_right[node], depth + 1)
#         else:
#             print("{}return {}".format(indent, tree_.value[node]))
#     recurse(0, 1)

In [30]:
scaler = StandardScaler()
scaler.fit(train)
train = pd.DataFrame(data=scaler.transform(train), columns=train.columns)
test = pd.DataFrame(data=scaler.transform(test), columns=test.columns)

In [31]:
full = pd.concat([train, test], axis=0)

pcamod = PCA(n_components=10, random_state=1)
pcamod.fit(full)
xtrain = pcamod.transform(train)
xtest = pcamod.transform(test)
trainfull = pd.concat([train, pd.DataFrame(xtrain)], axis=1, ignore_index=True)
testfull = pd.concat([test, pd.DataFrame(xtest)], axis=1, ignore_index=True)

In [33]:
cols=train.columns

for i in cols:
    newcol = str(i + '_bucket')
    mod = DecisionTreeClassifier(max_depth=2)
    mod.fit(train[i].to_frame(), train_res)
    train[newcol] = mod.predict_proba(train[i].to_frame())[:,1]
    test[newcol] = mod.predict_proba(test[i].to_frame())[:,1]

In [34]:
dummy_train, dummy_test, dummy_ytrain, dummy_ytest = train_test_split(train, train_res,
                                                    stratify=train_res,
                                                    test_size=0.3, random_state=1)

In [48]:
import xgboost as xgb

def get_xgb_imp(xgb):
    imp_vals = xgb.get_fscore()
    feats_imp = pd.DataFrame(imp_vals,index=np.arange(2)).T
    feats_imp.iloc[:,0]= feats_imp.index    
    feats_imp.columns=['feature','importance']
    feats_imp.sort_values('importance',inplace=True,ascending=False)
    feats_imp.reset_index(drop=True,inplace=True)
    return feats_imp

def quick_model(train, valid, ytrain, yvalid):
    xg_train = xgb.DMatrix(train, label=ytrain)
    xg_test = xgb.DMatrix(valid, label=yvalid)

    # setup parameters for xgboost
    param = {'objective':'binary:logistic', 'max_depth':50, 'silent':1, 'nthread':-1, 'subsample':0.75, 
             'colsample_bytree':0.8, 'learning_rate':0.1, 'eval_metric':['logloss', 'error'], 'n_jobs': -1,
            'colsample_bylevel':0.7, 'tree_method':'hist', 'seed':10, 'grow_policy':'lossguide', 'max_delta_step':3,
            'max_bin':200}

    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 100
    model = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=30, verbose_eval=10)
    # get prediction
    pred = model.predict(xg_test)
    error_rate = np.sum(pred != yvalid) / yvalid.shape[0]
    print('Test error using softprob = {}'.format(error_rate))

    print(skm.accuracy_score(y_pred=pred, y_true=yvalid))
    print(skm.confusion_matrix(y_pred=pred, y_true=yvalid))
    return model, feat_names, xg_train, xg_test, pred, pred_probs

In [ ]:
model, feat_names, xg_train, xg_test, pred, pred_probs = quick_model(dummy_train, dummy_test, dummy_ytrain, dummy_ytest)

In [20]:
X = np.array(dummy_train)
XV = np.array(dummy_test)
Y = to_categorical(dummy_ytrain)
YV = to_categorical(dummy_ytest)

In [21]:
Y.shape

(140000, 2)

In [22]:
input_dim = dummy_train.shape[1]

class_weight = {0: 1, 1: 1}

model = Sequential()
model.add(Dense(input_dim, input_dim = input_dim , activation = 'relu'))
model.add(BatchNormalization())
model.add(Input(shape = (2000,)))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(200, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(10, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(2, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
model.fit(X, Y, epochs = 3, batch_size = 50, class_weight=class_weight, shuffle=True)
scores = model.evaluate(XV, YV)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/3
140000/140000 [==============================] - 26s 189us/sample - loss: 0.2913 - accuracy: 0.8855
Epoch 2/3
140000/140000 [==============================] - 25s 175us/sample - loss: 0.2387 - accuracy: 0.9114
Epoch 3/3
60000/60000 [==============================] - 5s 85us/sample - loss: 0.2512 - accuracy: 0.9072

accuracy: 90.72%


In [23]:
pred=model.predict(XV)
sklearn.metrics.roc_auc_score(y_score=pred, y_true=YV)

0.8615023556924606

In [17]:
# X = np.array(xtrain)
# XV = np.array(xtest)
# Y = to_categorical(ytrain)
# YV = to_categorical(ytest)

X = np.reshape(xxtrain.values, (-1, 306, 1))
Y = ytrain
XV = np.reshape(xxtest.values, (-1, 306, 1))
YV = ytest

In [20]:
T = np.reshape(testfull.values, (-1, 300, 1))

pred=model.predict(T)
ss['target'] = pred

In [21]:
ss.to_csv('sub.csv', index=False)